In [8]:
!pip install praw
!pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 55.4 kB/s eta 0:00:0000:0500:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 46.8 kB/s eta 0:00:0000:0200:07
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [36]:
import praw
import pandas as pd
import datetime
import time

In [5]:
reddit = praw.Reddit(user_agent=True,
                     client_id="qxWM9jmK8cAIshLz8FIXpw",
                     client_secret="0qzOxNNEYGFUo_9oQEZDN42z3x_JEA",
                     username="pythonprojectmickey",
                     password="Nvk#pHLdsTS2Q74")

In [31]:
url = "https://www.reddit.com/r/apple/comments/1l7ewbh/wwdc_2025_postevent_megathread/"

post = reddit.submission(url=url)
# print(post.selftext)
comments = []
for comment in post.comments: 
    comments.append(comment.body)

print(len(comments))

91


In [32]:
# Define the subreddit
subreddit = reddit.subreddit("Siri")

# User Config
KEYWORDS = ['Intelligence', 'WWDC'] #keywords to search in title or body
POST_LIMIT = 100 #max num of posts to scan
TIME_FILTER = 'year'

# Extract
siri_posts = []

for post in subreddit.top(limit=POST_LIMIT, time_filter=TIME_FILTER): 
  title_lower = post.title.lower()
  selftext_lower = post.selftext.lower() if post.selftext else ''

  if any(keyword in title_lower or keyword in selftext_lower for keyword in KEYWORDS): 
    post.comments.replace_more(limit=0)
    top_comments = [comment.body for comment in post.comments[:5]]

    siri_posts.append({
        'Title': post.title,
        'Score': post.score,
        'URL': post.url,
        'Comments Count': post.num_comments,
        'Top Comments': top_comments
    })

print(siri_posts)

[]


In [ ]:
subreddit = reddit.subreddit("Siri")

In [37]:
datetime.datetime.now()

datetime.datetime(2025, 6, 16, 23, 15, 18, 957203)

In [ ]:
subreddit_name = "Siri"
start_date = datetime.datetime(2025, 6, 10, 1, 0, 0) #the time WWDC2025 started in HKT

start_timestamp = int(start_date.timestamp())
current_timestamp = int(datetime.datetime.now().timestamp())

subreddit = reddit.subreddit(subreddit_name)
posts = []
after = None

while True:
    try:
        if after:
            results = subreddit.new(limit=20, params={'after': after})
        else:
            results = subreddit.new(limit=20)

        for submission in results:
            if submission.created_utc < start_timestamp:
                break  # Stop if we've passed the start date

            if submission.created_utc <= current_timestamp:
                posts.append({
                    'title': submission.title,
                    'selftext': submission.selftext,
                    'author': str(submission.author),
                    'score': submission.score,
                    'created_utc': submission.created_utc,
                    # ... other data
                })
                after = f"t3_{submission.id}" # Get the ID of the current post
            else:
                after = None
                break #Stop if we've passed the end date
        
        if not after:
            break #Exit the loop if there are no more posts to fetch

        time.sleep(2) #Be nice to the API
    except Exception as e:
        print(f"An error occurred: {e}")
        break

print(f"Scraped {len(posts)} posts.")

An error occurred: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /r/Siri/new?after=t3_1l7bvav&limit=100&raw_json=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x117ba0a50>: Failed to resolve 'oauth.reddit.com' ([Errno 8] nodename nor servname provided, or not known)"))
Scraped 17 posts.


In [40]:
# Forming a Data Frame 
df = pd.DataFrame(posts)
df.head() 

,title,selftext,author,score,created_utc
0,“Message from someone”,"Hello, someone might be my saviour today, I lo...",Benmichael1989,2,1.750067e+09
1,"2025, Siri still cannot understand basic. Give...","Asks for iPad’s battery, replies with iPhone’s...",Exact_Confusion_5744,0,1.750067e+09
2,I can’t with this anymore 😂,,UrMom306,6,1.750002e+09
3,Stupidest Siri response of all time,"Back when I had my first iPhone, the 4S, I ask...",stevenjklein,11,1.750001e+09
4,"I asked Siri my address, now I can’t sleep. Wh...",Around 1:30 in the morning here. I am a teenag...,Present_Deal_1751,0,1.749969e+09


In [ ]:
# Saving data into a CSV file
df.to_csv('siri_reddit_reviews.csv', index=False)

In [ ]:
for submission in subreddit.new(limit=10):
    print(submission.title)
    print(submission.url)

“Message from someone”
https://www.reddit.com/r/Siri/comments/1lcol2z/message_from_someone/
2025, Siri still cannot understand basic. Gives me unreliable response. #ecosystemfails
https://i.redd.it/wu69w9mxe97f1.jpeg
I can’t with this anymore 😂
https://i.redd.it/56k65468147f1.jpeg
Stupidest Siri response of all time
https://www.reddit.com/r/Siri/comments/1lc2uln/stupidest_siri_response_of_all_time/
I asked Siri my address, now I can’t sleep. What odd responses has Siri ever given you?
https://www.reddit.com/r/Siri/comments/1lbtmpb/i_asked_siri_my_address_now_i_cant_sleep_what_odd/
Bitch, please!!
https://i.redd.it/3z73xju79s6f1.jpeg
As a new iPhone user, please help me understand a few things about Siri’s capabilities
https://www.reddit.com/r/Siri/comments/1l9xtip/as_a_new_iphone_user_please_help_me_understand_a/
Creating a Conversational AI like Samantha from the movie Her and Jarvis from Iron Man. How can something like this help you in your daily lives?
https://www.reddit.com/r/Siri

In [34]:
for submission in subreddit.top(limit=1):
    for comment in submission.comments:
        print(comment.body)

[“Siri and Apple's use of AI has been severely held back by caution and organizational dysfunction, according to over three dozen former Apple employees who spoke to The Information's Wayne Ma.”](https://www.macrumors.com/2023/04/27/report-details-turmoil-behind-siri-and-apple-ai/)
And what is the most hilarious is that she’s getting even worse
You can train it, along with accessibility to make it do much. It can open programs at least
I’m embedded in the ecosystem and I wish it was better. I feel the opposite about Siri than I do about their product suite.
I don't know what you mean by "How is Siri so useless in 2023". How about a web search for it?  


\[Search the web\]
It's been a decade and it can only speak 21 languages. Like, come on. Apple doesn't seem to care about half of the European countries despite the fact that many people around here have iPhones.
Siri is the Stacy Malibu of virtual assistants
Exactly one week ago, Siri celebrated her 12th birthday, which I had to cook 

In [35]:
for submission in subreddit.top(limit=1):
    for comment in submission.comments:
        print(comment.body)
        for reply in comment.replies:
            print(f"  > {reply.body}")

[“Siri and Apple's use of AI has been severely held back by caution and organizational dysfunction, according to over three dozen former Apple employees who spoke to The Information's Wayne Ma.”](https://www.macrumors.com/2023/04/27/report-details-turmoil-behind-siri-and-apple-ai/)
  > Great article, answered all my questions and more. Thank you!!
  > It’s like it’s run by Disney woke + and Wokeflix .  Afraid to respond to the wrong thing or say the wrong thing
And what is the most hilarious is that she’s getting even worse
  > ? 

Is she getting worse or is competitive ***actual*** ai getting better? 

I don’t think she’s gotten worse. I think she hasn’t been improved since 2010
You can train it, along with accessibility to make it do much. It can open programs at least
I’m embedded in the ecosystem and I wish it was better. I feel the opposite about Siri than I do about their product suite.
I don't know what you mean by "How is Siri so useless in 2023". How about a web search for it?

In [22]:
df = pd.DataFrame(siri_posts)

In [23]:
df

""
